In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [4]:
payment_data = pd.read_csv("/kaggle/input/brazilian-ecommerce/olist_order_payments_dataset.csv")
payment_data.head(10)

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45
5,298fcdf1f73eb413e4d26d01b25bc1cd,1,credit_card,2,96.12
6,771ee386b001f06208a7419e4fc1bbd7,1,credit_card,1,81.16
7,3d7239c394a212faae122962df514ac7,1,credit_card,3,51.84
8,1f78449c87a54faf9e96e88ba1491fa9,1,credit_card,6,341.09
9,0573b5e23cbd798006520e1d5b4c6714,1,boleto,1,51.95


In [5]:
payment_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB


In [6]:
# Alter all rows name where payment type = not_defined to cash
payment_data.loc[payment_data["payment_type"] == "not_defined", "payment_type"] = "cash"

In [7]:
# How many payment records are there in total?
total_records = payment_data.shape[0]
total_records

103886

In [8]:
# What are the different types of payment methods used in the dataset?
payment_data["payment_type"].unique()

array(['credit_card', 'boleto', 'voucher', 'debit_card', 'cash'],
      dtype=object)

In [9]:
# Which payment type appears most frequently?
payment_data["payment_type"].value_counts().index[0]

'credit_card'

In [10]:
# What is the total amount paid across all orders?
payment_data["payment_value"].sum().round(2)

16008872.12

In [11]:
# How many payments were made in a single installment?
payment_data[payment_data["payment_installments"] == 1].shape[0]

52546

In [12]:
# What is the average payment value for each payment type?
payment_data.groupby("payment_type")["payment_value"].mean().sort_values(ascending = False)

payment_type
credit_card    163.319021
boleto         145.034435
debit_card     142.570170
voucher         65.703354
cash             0.000000
Name: payment_value, dtype: float64

In [13]:
# How many times does each number of installments occur?
payment_data.groupby("payment_installments").size().sort_values(ascending = False)

payment_installments
1     52546
2     12413
3     10461
4      7098
10     5328
5      5239
8      4268
6      3920
7      1626
9       644
12      133
15       74
18       27
11       23
24       18
20       17
13       16
14       15
17        8
16        5
21        3
0         2
22        1
23        1
dtype: int64

In [14]:
# What is the total payment value per payment type?
payment_data.groupby("payment_type")["payment_value"].sum().sort_values(ascending = False)

payment_type
credit_card    12542084.19
boleto          2869361.27
voucher          379436.87
debit_card       217989.79
cash                  0.00
Name: payment_value, dtype: float64

In [15]:
# What is the highest and lowest payment value in the dataset?
payment_data["payment_value"].min(), payment_data["payment_value"].max()

(0.0, 13664.08)

In [16]:
# How many unique orders used more than one payment (look for repeated order_ids)?
payment_per_order = payment_data.groupby("order_id")["payment_type"].nunique().reset_index(name = "payment_method_count")
payment_per_order = len(payment_per_order[payment_per_order["payment_method_count"] > 1])
payment_per_order

2246

In [17]:
# Which orders had the most number of installments? List top 10?
installment_per_order = (
    payment_data
    .groupby("order_id")["payment_installments"]
    .sum()
    .reset_index(name = "installment_count")
    .sort_values("installment_count", ascending = False)
)
installment_per_order.head(10)

,order_id,installment_count
97261,fa65dad1b0e818e3ccc5cb0e39231352,29
79611,ccf804e764ed5650cd8759557269dc13,26
16742,2b7dbe9be72b8f9733844c31055c0825,25
56015,90f864fe19d11549fa01eb81c4dd87e3,25
93069,ef71772d55431467890fda2f45c7bdde,24
41502,6ae2e8b8fac02522481d2a2f4ca4412c,24
95551,f60ce04ff8060152c83c7c97e246d6a8,24
84333,d8d5cc8b2d42cce90b7ea35e5691a7b1,24
83751,d74fca7ee2ce7587c45eefb0fea95ed8,24
9290,1800f4a4362310d7eb38281634f6dfd1,24


In [18]:
# What’s the average number of installments for each payment?
avg_installment_per_type = (
    payment_data
    .groupby("payment_type")["payment_installments"]
    .mean()
    .reset_index(name = "avg_installment_count")
    .sort_values("avg_installment_count", ascending = False)
)
avg_installment_per_type.round(2)

,payment_type,avg_installment_count
2,credit_card,3.51
0,boleto,1.00
1,cash,1.00
3,debit_card,1.00
4,voucher,1.00


In [19]:
# Identify the top 5 orders with the highest total payment value, considering that an order may have multiple payment rows.
total_payment_value_order = (
    payment_data
    .groupby("order_id")["payment_value"]
    .sum()
    .reset_index(name = "total_payment_value")
    .sort_values("total_payment_value", ascending = False)
)
total_payment_value_order.head(5)

,order_id,total_payment_value
1471,03caa2c082116e1d31e67e9ae3700499,13664.08
44830,736e1922ae60d0d6a89247b851902527,7274.88
3156,0812eb902a67711a1cb742b3cdaa65ae,6929.31
99071,fefacc66af859508bf1a7934eab1e97f,6922.21
95186,f5136e38d1a14a4dbd87dff67da82701,6726.66


In [20]:
# Which payment type has the highest average number of installments?
payment_data.groupby("payment_type")["payment_value"].mean().sort_values(ascending = False).index[0]

'credit_card'

In [21]:
# What percentage of total payments was made using credit card vs boleto?
total_payment = payment_data["payment_value"].sum()
credit_card_perc = (payment_data[payment_data["payment_type"] == "credit_card"]["payment_value"].sum() / total_payment) * 100
boleto_perc = (payment_data[payment_data["payment_type"] == "boleto"]["payment_value"].sum() / total_payment) * 100
credit_card_perc, boleto_perc

(78.34458352834915, 17.923569183960726)